In [32]:
# Import necessary libraries for data manipulation and machine learning
import pandas as pd  # Library for data manipulation and analysis
import numpy as np  # Library for numerical operations
import tensorflow  # Library for machine learning
import keras  # Library for deep learning
from sklearn.preprocessing import LabelEncoder  # Library for encoding categorical data
from sklearn.model_selection import train_test_split  # Library for splitting data into training and testing sets
from keras.models import Sequential  # Library for building sequential neural networks
from keras.layers import Embedding, Flatten, Dense  # Libraries for building neural network layers
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Library for padding sequences
from tensorflow.keras.preprocessing.text import Tokenizer  # Library for tokenizing text data

import warnings
warnings.filterwarnings('ignore')



In [33]:
data = pd.read_csv('train.txt', sep=';')
data.columns =['Text', 'Emotions']
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [34]:
# Tokenize the text data to prepare it for model training
texts = data['Text'].tolist()
labels = data['Emotions'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [35]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [36]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [37]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [38]:
one_hot_labels = keras.utils.to_categorical(labels)

In [39]:
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

In [40]:
#define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_length))

model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation='softmax'))

model.compile(optimizer = 'adam', loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
359/400 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.3710 - loss: 1.5442

KeyboardInterrupt: 

In [47]:
input_text = "i visited park and playland"

#

input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction=model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
joy
